In [4]:
import requests
from bs4 import BeautifulSoup

def get_all_raw_url() :
    urls_brutes = []

    for i in range (0, 286) :
        urls_brutes.append(f"https://www.chambredesrepresentants.ma/fr/actualite?page={i}")
    return urls_brutes

def get_all_url (url) :

    r = requests.get(url)
    bs = BeautifulSoup(r.content, "html.parser")
    container_url = bs.find_all('div', class_='col-12')
    urls = []
    list_a = []

    for e in container_url :
        list_a.append(e.a)

    for e in list_a :
        urls.append(e.get('href'))

    lien_base = "https://www.chambredesrepresentants.ma"
    urls_final = []

    for i in range (len(urls)):

        try:
             if requests.get(lien_base + urls[i]).status_code == 200 :
                 urls_final.append(lien_base+urls[i])
        except requests.exceptions.RequestException as e:
            continue

    return urls_final

In [5]:
import requests
from bs4 import BeautifulSoup

def get_data(url):
    
    try:
        r = requests.get(url)
        bs = BeautifulSoup(r.content, "html.parser")
        list_element = []
        list_element.append(bs.h1.text)

        container_date = bs.find('span', class_='date-news d-block')
        list_element.append(container_date.text)

        container_div =bs.find('div', class_='field-item even')
        list_span = []

        for p in container_div.find_all('p'):
                list_span.append(p.text)
        list_element.append(list_span)

        return list_element
    
    except Exception as e:
        print(f"erreur dans le lien suivant : {url} ")
        return None

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import keyboard

raw_url = get_all_raw_url()
urls = []

for e in raw_url:
    url_tempo = []
    url_tempo = get_all_url(e)
    for j in url_tempo :
        urls.append(j)

df = pd.DataFrame(columns=['Title', 'Time', 'Content'])


for t in urls :
    
    if keyboard.is_pressed('s'):
        print("Pause sur demande (touche 's' enfoncée).")
        break
    try:
        if get_data(t) == None :
            nouvelle_ligne = pd.Series([None,None, None], index=['Title', 'Time', 'Content'])
        
        else:   
            tempo = get_data(t)
            text = '\n'.join(tempo[2])
    
            nouvelle_ligne = pd.Series([tempo[0], tempo[1], text], index=['Title', 'Time', 'Content'])
    
        df = pd.concat([df, nouvelle_ligne.to_frame().T], ignore_index=True)
        
    except Exception as e:  
        print("erreur sur le site {t}")
df

erreur dans le lien suivant : https://www.chambredesrepresentants.ma/fr/visites-historiques/le-souverain-du-royaume-hachemite-de-jordanie-recoit-le-chef-du-gouvernement-et 
erreur dans le lien suivant : https://www.chambredesrepresentants.ma/fr/activites-diplomatiques/le-president-du-conseil-representatif-des-regions-dindonesie-appelle-au 
erreur dans le lien suivant : https://www.chambredesrepresentants.ma/fr/activites-diplomatiques/atelier-des-parlements-de-lasie-du-sud-sur-les-odd-jakarta-avec-la 
erreur dans le lien suivant : https://www.chambredesrepresentants.ma/fr/seminaires-et-conferences/lancement-rabat-du-reseau-des-femmes-parlementaires-arabes-pour-legalite 
erreur dans le lien suivant : https://www.chambredesrepresentants.ma/fr/actualites/le-renforcement-des-relations-parlementaires-au-centre-des-entretiens-de-m-talbi-alami-et 
erreur dans le lien suivant : https://www.chambredesrepresentants.ma/fr/activites-diplomatiques/agenda-diplomatique-de-la-chambre-des-representants-

,Title,Time,Content
0,La Chambre des représentants a adopté 44 proje...,13/02/2019,La Chambre des représentants a adopté 44 proje...
1,La présidente du parlement de Cantabrie salue ...,13/02/2019,"La présidente du parlement de Cantabrie, Dolor..."
2,Une délégation de femmes parlementaires en vis...,12/02/2019,Une délégation de femmes parlementaires effect...
3,La Chambre des Représentants adopte 4 projets ...,12/02/2019,La Chambre des Représentants a adopté 4 projet...
4,"M. Habib El Malki, Président de la Chambre des...",08/02/2019,"Le Président de la Chambre des représentants, ..."
...,...,...,...
1344,M.Ghellab s'entretient avec le président de la...,26/06/2012,"Le président de la Chambre des représentants, ..."
1345,Le chef de la diplomatie sénégalaise qualifie ...,26/06/2012,Le ministre sénégalais des Affaires étrangères...
1346,M. Ghellab participe à Bruxelles à une confére...,26/06/2012,"Le président de la Chambre de représentants, K..."
1347,Le chef d'une délégation parlementaire tanzani...,26/06/2012,Le président de la Commission des affaires étr...


In [ ]:
df

In [7]:
import os

# Spécifiez le dossier cible
dossier_cible = 'D:/Etude/insea/3eme année/Big Data'

# Spécifiez le chemin complet du fichier SQLite, en utilisant os.path.join pour combiner le dossier cible et le nom du fichier
chemin_fichier_sqlite = os.path.join(dossier_cible, 'raw_data.db')

# Enregistrez le DataFrame dans le fichier SQLite
df.to_sql('raw_data', f'sqlite:///{chemin_fichier_sqlite}', index=False, if_exists='replace')


1349

In [5]:
pip install keyboard

                                              0.0/58.1 kB ? eta -:--:--
     ---------------------------------------- 58.1/58.1 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
